In [22]:
import pandas as pd
from geopy.distance import geodesic
import datetime

In [32]:
path="Dataset/cleaned_data.csv"
df= pd.read_csv(path)
df.head()

,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28
2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23
3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36


In [33]:
df.shape

(534368, 5)

In [19]:
def calculate_distance(row):
    origin = (float(row['Trip Origin'].split(',')[0]), float(row['Trip Origin'].split(',')[1]))
    destination = (float(row['Trip Destination'].split(',')[0]), float(row['Trip Destination'].split(',')[1]))
    distance = geodesic(origin, destination).miles
    return distance

df = pd.DataFrame({
    'Trip ID': [391996, 391997, 391998, 391999, 392001],
    'Trip Origin': ['6.508813001668548,3.37740316890347', '6.4316714,3.4555375', '6.631679399999999,3.3388976', '6.572757200000001,3.3677082', '6.6010417,3.2766339'],
    'Trip Destination': ['6.650969799999999,3.3450307', '6.4280814653326,3.4721885847586', '6.508324099999999,3.3590397', '6.584881099999999,3.3614073', '6.4501069,3.3916154'],
    'Trip Start Time': ['2021-07-01 07:28:04', '2021-07-01 06:38:04', '2021-07-01 06:21:02', '2021-07-01 07:16:07', '2021-07-01 09:30:59'],
    'Trip End Time': ['2021-07-01 07:29:37', '2021-07-01 07:07:28', '2021-07-01 07:02:23', '2021-07-01 07:29:42', '2021-07-01 09:34:36']
})

df['Distance'] = df.apply(lambda row: calculate_distance(row), axis=1)

print(df)

   Trip ID                         Trip Origin  \
0   391996  6.508813001668548,3.37740316890347   
1   391997                 6.4316714,3.4555375   
2   391998         6.631679399999999,3.3388976   
3   391999         6.572757200000001,3.3677082   
4   392001                 6.6010417,3.2766339   

                  Trip Destination      Trip Start Time        Trip End Time  \
0      6.650969799999999,3.3450307  2021-07-01 07:28:04  2021-07-01 07:29:37   
1  6.4280814653326,3.4721885847586  2021-07-01 06:38:04  2021-07-01 07:07:28   
2      6.508324099999999,3.3590397  2021-07-01 06:21:02  2021-07-01 07:02:23   
3      6.584881099999999,3.3614073  2021-07-01 07:16:07  2021-07-01 07:29:42   
4              6.4501069,3.3916154  2021-07-01 09:30:59  2021-07-01 09:34:36   

    Distance  
0  10.018648  
1   1.170853  
2   8.588837  
3   0.938913  
4  13.039051  


In [34]:
df.shape

(534368, 5)

In [20]:

def calculate_distance(row):
    origin = (float(row['Trip Origin'].split(',')[0]), float(row['Trip Origin'].split(',')[1]))
    destination = (float(row['Trip Destination'].split(',')[0]), float(row['Trip Destination'].split(',')[1]))
    distance = geodesic(origin, destination).miles
    return distance

def calculate_speed(row):
    distance = row['Distance']
    start_time = pd.to_datetime(row['Trip Start Time'])
    end_time = pd.to_datetime(row['Trip End Time'])
    time_diff = end_time - start_time
    speed = distance / time_diff.total_seconds() * 3600
    return speed

df = pd.DataFrame({
    'Trip ID': [391996, 391997, 391998, 391999, 392001],
    'Trip Origin': ['6.508813001668548,3.37740316890347', '6.4316714,3.4555375', '6.631679399999999,3.3388976', '6.572757200000001,3.3677082', '6.6010417,3.2766339'],
    'Trip Destination': ['6.650969799999999,3.3450307', '6.4280814653326,3.4721885847586', '6.508324099999999,3.3590397', '6.584881099999999,3.3614073', '6.4501069,3.3916154'],
    'Trip Start Time': ['2021-07-01 07:28:04', '2021-07-01 06:38:04', '2021-07-01 06:21:02', '2021-07-01 07:16:07', '2021-07-01 09:30:59'],
    'Trip End Time': ['2021-07-01 07:29:37', '2021-07-01 07:07:28', '2021-07-01 07:02:23', '2021-07-01 07:29:42', '2021-07-01 09:34:36']
})

df['Distance'] = df.apply(lambda row: calculate_distance(row), axis=1)

df['Speed'] = df.apply(lambda row: calculate_speed(row), axis=1)


In [21]:

def get_weekday_or_weekend(date):
    day = date.weekday()
    if day == 0 or day == 6:
        return 'Weekend'
    else:
        return 'Weekday'


In [23]:
df['Day'] = df['Trip Start Time'].apply(lambda x: get_weekday_or_weekend(pd.to_datetime(x).date()))

In [27]:
# Simplified weather classification
weather_classification = {
    "clear sky": "Sunny",
    "few clouds": "Sunny",  # Assuming few clouds still make for a generally sunny day
    "scattered clouds": "Cloudy",
    "broken clouds": "Cloudy",
    "shower rain": "Rainy",
    "rain": "Rainy",
    "thunderstorm": "Rainy",  # Thunderstorms typically involve rain
    "snow": "Rainy",         # Treating snow as rainy for simplicity, adjust as needed
    "mist": "Cloudy",        # Mist can be considered a form of cloudiness
    # Add more mappings or adjust existing ones as per your preference
}

In [29]:
import requests
from datetime import datetime

def get_weather_simplified(date):
    api_key = "a7b70f6e-27e2-11ef-9acf-0242ac130004-a7b7102c-27e2-11ef-9acf-0242ac130004"
    base_url = "http://api.openweathermap.org/data/2.5/forecast"
    params = {
        "q": "Lagos",
        "appid": api_key,
        "units": "imperial"
    }
    
    response = requests.get(base_url, params=params)
    weather_data = response.json()
    
    classified_forecasts = []
    for forecast in weather_data["list"]:
        forecast_date = datetime.utcfromtimestamp(forecast["dt"]).date()
        if forecast_date == date:
            description = forecast["weather"][0]["description"]
            # Simplified classification
            weather_type = weather_classification.get(description, "Unknown")
            classified_forecasts.append({"date": forecast_date, "description": description, "weather_type": weather_type})
    
    return classified_forecasts

In [25]:
import holidays
import datetime

# Define the holidays for Nigeria
ng_holidays = holidays.NG()

def get_weekday_or_weekend(date):
    day = date.weekday()
    if day == 0 or day == 6:
        return 'Weekend'
    else:
        return 'Weekday'

def check_holiday(date):
    return date in ng_holidays

In [ ]:
cleaned_df =df['Distance'] 